Training durch verschlüsselte Daten
===

Wenn ein Modell beziehungsweise dessen Gewichte trainiert werden sollen wird das Ergebnis eines Forward Pass verwendet. Dies bedeutet, dass auch die Ableitung der Gewichte ebenfalls verschlüsselt sein wird und somit auch die Gewichte verschlüsselt sind. Das gesamte Modell muss also ebenfalls mit dem Schlüssel des Datenbesitzers verschlüsselt werden.

Die vorherig verwendete Architektur kann nicht direkt übernommen und muss angepasst werden. Hierfür gibt es verschiedene Möglichkeiten:
* Problem für das Training ist die letzte Sigmoid Schicht im Modell für die Inferenz, diese könnte zum Beispiel durch eine Approximation ersetzt werden. Dies würde bedeuten, dass der Kontext für die Verschlüsselung sehr groß sein muss oder aber die verschlüsselten Objekte oft neu verschlüsselt werden (bzw. Bootstrapping)
* Die Sigmoid Funktion muss aber nicht unbedingt eingesetzt werden in der letzten Schicht. Es kann auch eine weitere Quadratische Funktion angewendet werden auf die Ausgabe der Schicht. Dies sorgt dafür, dass ein Forward Pass durchgeführt werden kann ohne einen größeren Kontext zu brauchen als bei der Inferenz auf verschlüsselten Daten.

Interaktive Variante
---
Die vielen Multiplikationen während dem Training sorgt dafür, dass ein großer Kontext benötigt wird und somit die verschlüsselten Daten sehr viel Platz im Arbeitsspeicher einnehmen und ebenfalls lange für die Berechnung benötigen. Dies ist ab einer bestimmten Multiplikationstiefe nicht mehr effizient, wodurch der Leveled HE Ansatz nicht durchführbar ist. Hierfür sollte also ein Fully HE Ansatz verwendet werden, in welchem das Rauschen der verschlüsselten Objekte öfter zurückgesetzt werden kann. Dies könnte man auch im Leveled HE Ansatz machen, würde aber dazu führen, dass der Besitzer der Daten sehr oft Werte neu verschlüsseln müsste. Dieser Ansatz wird in diesem Beispiel umgesetzt.

In [102]:
import numpy as np
import matplotlib.pyplot as plt
import tenseal as ts
import time

X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
Y = np.array([[0, 1, 1, 0]])

def square(x):
    return np.square(x)

def square_prime(x):
    return 2 * x

w1_Training = np.random.rand(2,2)
w2_Training = np.random.rand(1,2)
b1_Training = np.zeros((2, 1))
b2_Training = np.zeros((1, 1))

learning_rate = 0.1

Für das Beispiel wird ein kleiner Kontext gewählt, damit die verschlüsselten Objekte nicht zu groß werden. Außerdem ist die Methode zur erneuten Verschlüsselung recrypt mit einer Wartezeit versehen, um die Kommunikation zu simulieren.

In [103]:
poly_mod_degree = 2 ** 13
bits_scale = 40

coeff_mod_bit_sizes=[60, bits_scale, bits_scale, 60]
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

ctx_training.global_scale = pow(2, bits_scale)
ctx_training.generate_galois_keys()

def encrypt(matrix):
    return np.array([[ts.ckks_vector(ctx_training, [value]) for value in column] for column in matrix])

def decrypt(matrix):
    return np.array([[column.decrypt()[0] for column in row] for row in matrix])

def recrypt(matrix):
    time.sleep(0.1)
    e_value = decrypt(matrix)
    return encrypt(e_value)

encrypted_X = encrypt(X)
encrypted_Y = encrypt(Y)

w1_Training = encrypt(w1_Training)
w2_Trianing = encrypt(w2_Training)
b1_Training = encrypt(b1_Training)
b2_Training = encrypt(b2_Training)

Im Forward Pass muss durch den Kontext nur nach jeder Aktivierungsfunktion die Daten neu verschlüsselt werden.

In [104]:
def forward_Training(w1, w2, x):
    z1 = np.add(np.dot(w1, x), b1_Training)
    a1 = square(z1)

    a1 = recrypt(a1)

    z2 = np.add(np.dot(w2, a1), b2_Training)
    a2 = square(z2)
    
    a2 = recrypt(a2)
    
    return z1, a1, z2, a2

Ähnlich wie im Forward Pass muss nicht jeder Wert direkt wieder neu verschlüsselt werden, hauptsächlich die Werte, welche weitergegeben werden an vorherige Schichten.

In [105]:
def backward_Training(w1, w2, z1, a1, z2, a2, x, y):
    dz2 = a2 - y
    dw2 = np.dot(dz2, a1.T)
    db2 = np.sum(dz2, axis = 1, keepdims = True)
    da1 = np.dot(w2.T, dz2)
    da1 = recrypt(da1)

    a_error = square_prime(z1)
    a_error = recrypt(a_error)
    dz1 = np.multiply(da1, a_error)
    dz1 = recrypt(dz1)
    dw1 = np.dot(dz1, x.T)
    db1 = np.sum(dz1, axis = 1, keepdims = True)

    return dw1, db1, dw2, db2

Das Training ist fast gleich wie das unverschlüsselte Training. Der einzige Unterschied ist, dass nach dem Verändern der Gewichte es wichtig ist diese wieder neu zu verschlüsseln.

In [106]:
iterations = 100
for i in range(iterations):
    z1, a1, z2, a2 = forward_Training(w1_Training, w2_Training, encrypted_X)

    dw1, db1, dw2, db2 = backward_Training(w1_Training,w2_Training,z1,a1,z2,a2,encrypted_X,encrypted_Y)

    w2_Training = w2_Training - learning_rate * dw2
    w1_Training = w1_Training - learning_rate * dw1
    b2_Training = b2_Training - learning_rate * db2
    b1_Training = b1_Training - learning_rate * db1

    w2_Training = recrypt(w2_Training)
    w1_Training = recrypt(w1_Training)
    b2_Training = recrypt(b2_Training)
    b1_Training = recrypt(b1_Training)

Nach dem verschlüsselten Training können die Gewichte wieder entschlüsselt werden. Dadurch kann die Inferenz effizienter durchgeführt werden oder falls der Datenbesitzer dies nicht möchte kann auch auf einem verschlüsselten Modell Inferenz durchgeführt werden.

In [107]:
result_w1_Training = decrypt(w1_Training)
result_w2_Training = decrypt(w2_Training)
result_b1_Training = decrypt(b1_Training)
result_b2_Training = decrypt(b2_Training)

Für die Inferenz muss noch eine weitere Funktion erstellt werden, da nicht angenommen wird, dass die Eingaben neu verschlüsselt werden müssen.

In [108]:
def crypted_forward(w1, w2, b1, b2, x):
    z1 = np.add(np.dot(w1, x), b1)
    a1 = square(z1)
    z2 = np.add(np.dot(w2, a1), b2)
    z3 = square(z2)
    return z3   

In [109]:
def predict(w1,w2,b1,b2,input):
    a2 = crypted_forward(w1,w2,b1,b2,test)
    a2 = np.squeeze(a2)
    if a2>=0.5:
        print([i[0] for i in input], " --> 1")
    else:
        print([i[0] for i in input], " --> 0")

test = np.array([[0],[0]])
predict(result_w1_Training,result_w2_Training,result_b1_Training,result_b2_Training,test)
test = np.array([[0],[1]])
predict(result_w1_Training,result_w2_Training,result_b1_Training,result_b2_Training,test)
test = np.array([[1],[0]])
predict(result_w1_Training,result_w2_Training,result_b1_Training,result_b2_Training,test)
test = np.array([[1],[1]])
predict(result_w1_Training,result_w2_Training,result_b1_Training,result_b2_Training,test)

[0, 0]  --> 0
[0, 1]  --> 1
[1, 0]  --> 1
[1, 1]  --> 0


Solange die Methode recrypt nicht mit einer Wartezeit versehen ist, ist es schneller mit einem kleineren Kontext die Gewichte zu trainieren. Dies ändert sich aber sobald eine Wartezeit nach dem Aufruf der Methode eingebaut wird. Dies kommt von der erhöhten Anzahl der notwendigen erneuten Verschlüsselungen. Dies bedeutet, dass es ratsam ist eine Balance zu finden, damit die Berechnungen nicht zu lange dauern und wenig Kommunikation mit dem Datenbesitzer stattfindet.